# Project 4 

Outline:

* scraping data

* binning: by salary

* parsing data: tokenizing, bigrams, most_common words for each bin 

* building model

* evaluating model


Downloading the whole data for US. 09/29/2016

In [ ]:
df

In [128]:
# check one entry to verify contents
print df.iloc[6]

company                                                             Cisco Systems, Inc.
job_link        http://www.indeed.com/rc/clk?jk=6cb402d057d00c35&fccid=dfc44f3b8c44a6db
job_location                                                               San Jose, CA
job_posted                                                                  Just posted
job_title                                       Data Scientist - Technology and Quality
Name: 6, dtype: object


In [129]:
# Save the result to CSV
df.to_csv('./indeed_companies.csv', encoding='utf-8')

Downloading the whole data for Australia. 09/29/2016

In [130]:
# load the library


df_au = pd.DataFrame()   # create a new data frame
pd.set_option('max_colwidth',500)    # to remove column limit (default is 50. Otherwise, we'll lose some info)


# indeed.com's query url
base_url = 'http://au.indeed.com/jobs?q=Data+Scientist&jt=fulltime'
sort_by = 'date'          # sort by data
start_from = '&start='    # start page number

for page in range(1,10): # page from 1 to 100 
    page = (page-1) * 10   # pagination
    url = "%s%s%s%d" % (base_url, sort_by, start_from, page) # get full url 
    contents = BeautifulSoup(urllib2.urlopen(url), "lxml") 

    contentsElements_au = contents.find_all('div', attrs={"class" : '  row  result'})
    cont_au = contents.find('div', attrs = {'class': 'lastRow  row  result'}) ##just added last job on each page
    contentsElements_au.append(cont_au)
    # each row contains 1 job

    # extract job information
    for elem in contentsElements_au: 
        try:
            company_au = elem.find('span', attrs={'itemprop':'name'}).getText().strip() #strips off whitespace 
        except AttributeError:
            company = '-'
        job_title_au = elem.find('a', attrs={'class':'turnstileLink'}).attrs['title']
        home_url_au = "http://www.indeed.com"
        job_link_au = "%s%s" % (home_url,elem.find('a').get('href'))
        job_addr_au = elem.find('span', attrs={'itemprop':'addressLocality'}).getText()
        job_posted_au = elem.find('span', attrs={'class': 'date'}).getText()

           # add a record of job info to our data frame

        df_au = df_au.append({'company': company_au, 
                        'job_title': job_title_au, 
                        'job_link': job_link_au, 
                        'job_posted': job_posted_au,
                        'job_location': job_addr_au
                       }, ignore_index=True)
df_au

Save the result to CSV
df_au.to_csv('./indeed_companies_au.csv', encoding='utf-8')

jobs_data = pd.read_csv('indeed_companies.csv')
jobs_data_au = pd.read_csv('indeed_companies_au.csv')
frames = [jobs_data, jobs_data_au]
result = pd.concat(frames, axis = 0, ignore_index = True)
result

In [155]:
import matplotlib as plt
%matplotlib inline

.

In [51]:
def extract_text(el):
    if el:
        return el.text.strip()
    else:
        return ''      

def get_company_from_result(result):
    return extract_text(result.find('span', {'class' : 'company'}))

def get_location_from_result(result):
    return extract_text(result.find('span', {'class' : 'location'}))

def get_summary_from_result(result):
    return extract_text(result.find('span', {'class' : 'summary'}))

def get_title_from_result(result):
    return result.find('a', {'data-tn-element' : 'jobTitle'}).text.strip()

def get_salary_from_result(result):
    salary_table = result.find('td', {'class' : 'snip'})
    if salary_table:
        snip = salary_table.find('nobr')
        if snip:
            return snip.text.strip()

    return None


def extract_salary_average(salary_string):
    regex = r'\$([0-9]+,[0-9]+)'
    matches = re.findall(regex, salary_string)
    return np.mean([float(salary.replace(',', '')) for salary in matches ])


In [74]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib2, requests


url_template = "http://www.indeed.com/jobs?q=data+scientist+${}%2C000&l=USA&start={}"
max_results_per_city = 5000 # Set this to a high-value (5000) to generate more results. 

# Crawling more results, will also take much longer. First test your code on a small number of results and then expand.

#city = elem.find('span', attrs={'itemprop':'addressLocality'}).getText()

rows = []
for salary in set([ '55-70', '70-85', '85-100', '100-115']):
    for start in range(1,10): 
        r = requests.get(url_template.format(salary, start))
        soup = BeautifulSoup(r.content) 
        results = soup.findAll('div', { "class" : "result" })
        for result in results:
            if result:
                row = {}
                row['title'] = get_title_from_result(result)
                row['company'] = get_company_from_result(result)
                row['summary'] = get_summary_from_result(result)
                row['city'] = get_location_from_result(result) 
                row['bin'] = salary
            #row['salary'] = get_salary_from_result(result)
                rows.append(row)
                

In [5]:
#result.to_csv('./indeed_companies.csv', encoding='utf-8')

In [75]:
pd.set_option('max_colwidth',500)
result = pd.DataFrame.from_records(rows)
#result.set_option('max_colwidth',500) # create a new data frame
result.drop_duplicates(subset = "summary")

,bin,city,company,summary,title
0,85-100,"Santa Clara Valley, CA",Apple,"We are looking for a highly motivated data scientist proficient in generating meaningful insights from complex, multi-dimensional data that can inform and...",Data Scientist - Fitness & Health
1,85-100,"Santa Clara Valley, CA",Apple,"Use distributed data processing and analysis. 5+ years experience working on data science projects. Attention to detail, data accuracy, and quality of output....",Data Scientist - Maps POI
2,85-100,"Washington, DC",Social Tables,"As Social Tables first Data Scientist, you'll be helping us to double-down on our data-informed culture. Skilled in SQL (and NoSQL experience is a plus) to get...",Data Scientist
3,85-100,"San Bruno, CA 94066",Walmart eCommerce,They assist in building large data sets from multiple sources in order to build algorithms for predicting future data characteristics....,Data Scientist Intern 2017- Grace Hopper
4,85-100,"Township of Piscataway, NJ",RangTech,"Experience in data visualization and presentation. Experience/Interest in machine learning, data mining, and predictive analysis....",Data Scientist
5,85-100,"Mountain View, CA",Upwork,"We need data scientists to explore, analyze, and leverage our data to build cutting edge data products for our marketplace....",Data Scientist Intern - Summer 2017
6,85-100,"Tampa, FL",ISPA Technology,"Seeking a Data Scientist to support the Intelligence Community and at Special Operations Command. Ability to merge data sources together, ensure consistency of...",Data Scientist
7,85-100,"Sunnyvale, CA 94089",Proofpoint,"An inquisitive mind with he ability to discover what the data is saying, using the various tools available to the data scientist....",Data Scientist
8,85-100,Remote,Crossover,We are looking for a highly experienced Applied Data Scientist enabling and owning lead scoring methods to drive profitable sales growth. About the Company....,Lead Scoring/Data Scientist - $85K
9,85-100,"Foster City, CA",Eurofins,"QC Data Review Scientist responsibilities include, but are not limited to, the following :. Enter and review analytical data in LIMS and ensure data meets...",QC Data review scientist


.

In [76]:
pd.set_option('max_colwidth',500)
df = pd.read_csv('indeed_companies.csv')
df[df.bin =="55-70"]

,Unnamed: 0,bin,city,company,summary,title
29970,29970,55-70,"Rockville, MD",ICF,"Experience analyzing data from large social surveys.*. We are business analysts, policy specialists, technologists, researchers, digital strategists, social...",Data Analyst
29971,29971,55-70,"Wellesley, MA 02481",Aetna,"The Data Scientist will develop, implement and measure the impact of solutions spanning all available touch points with health care consumers....",Data Scientist Statistician
29972,29972,55-70,"Newton, MA","Thermalin Diabetes, LLC",Knack for communicating stories through data visualizations. Format and analyze data to extract key performance parameters....,"Data Scientist - Newton, MA (will work in Boston, MA area)"
29973,29973,55-70,"Framingham, MA",Staples,: Qualifications Job : Location(s) : Other Locations : Schedule : : : ...,"Data Scientist, Framingham, MA"
29974,29974,55-70,"Hartford, CT 06156",Aetna,"Ability to work with large data sets from multiple data sources.· Demonstrates proficiency in several areas of data modeling, machine learning algorithms,...",Data Scientist Analyst
29975,29975,55-70,"Cambridge, MA",KAYAK.com,"Investigate data problems, identify patterns. This person should love working on data parsing, analyzing, investigating, interpreting, modeling, and extracting...",Data Scientist
29976,29976,55-70,"Springfield, VA",Concurrent Technologies Corporation,"Associate Data Scientist. CTC has an opportunity for a qualified Associate Data Scientist at Springfield, VA. Responsibilities include preparing thorough, well...",Associate Data Scientist
29977,29977,55-70,"Raleigh, NC","Resolvit, LLC","Technical knowledge regarding data models, database design development, data mining, and segmentation techniques....",Data Scientist
29978,29978,55-70,"Cambridge, MA",Massachusetts General Hospital(MGH),"In addition, this position involves data analysis and preparation of academic manuscripts and figures as well as abstracts to attend to scientific conferences....","Research Scientist, Sr. (Phd)"
29979,29979,55-70,"Alpharetta, GA",krg technology inc,Data scientist with either a Masters or PhD. Should know Machine learning....,Data Scientist


In [6]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39960 entries, 0 to 39959
Data columns (total 5 columns):
bin        39960 non-null object
city       39960 non-null object
company    39960 non-null object
summary    39960 non-null object
title      39960 non-null object
dtypes: object(5)
memory usage: 1.5+ MB


In [7]:
%matplotlib inline
import seaborn
result.bin.value_counts()

85-100     9990
55-70      9990
100-115    9990
70-85      9990
Name: bin, dtype: int64

In [223]:
#result.company.unique()

In [8]:
import nltk
import nltk.stem
import nltk.stem.porter
import nltk.stem.snowball
snowball = nltk.stem.snowball.SnowballStemmer("english")
from nltk.corpus import stopwords
import re

Extracting data from DataFrame and Cleaning it from numbers, charactters, stopwords.

In [213]:
junior = str(df[df['bin'].isin(['55-70'])].summary) #subsetting over categories(bins)
mid_low = str(df[df['bin'].isin(['70-85'])].summary)
mid_high = str(df[df['bin'].isin(['85-100'])].summary)
senior = str(df[df['bin'].isin(['100-115'])].summary)

junior_clean = re.sub("[^a-zA-Z]", " ", junior)
mid_low_clean = re.sub("[^a-zA-Z]", " ", mid_low)
mid_high_clean = re.sub("[^a-zA-Z]", " ", mid_high)
senior_clean = re.sub("[^a-zA-Z]", " ", senior)

#junior_clean

stop = set(stopwords.words('english'))
stop.update([',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}', '...', 'Data', 'data', 'scientist', 'Scientist'])

junior_words = [w for w in (nltk.word_tokenize(junior_clean)) if w.lower() not in stop]
mid_low_words = [w for w in (nltk.word_tokenize(mid_low_clean)) if w.lower() not in stop]
mid_high_words = [w for w in (nltk.word_tokenize(mid_high_clean)) if w.lower() not in stop]
senior_words = [w for w in (nltk.word_tokenize(senior_clean)) if w.lower() not in stop]


len(junior_words)

727

In [214]:
junior_words

['Experience',
 'analyzing',
 'large',
 'social',
 'surveys',
 'business',
 'analysts',
 'policy',
 'specialists',
 'technologists',
 'researchers',
 'digital',
 'strategists',
 'social',
 'develop',
 'implement',
 'measure',
 'impact',
 'solutions',
 'spanning',
 'available',
 'touch',
 'points',
 'health',
 'care',
 'consumers',
 'Knack',
 'communicating',
 'stories',
 'visualizations',
 'Format',
 'analyze',
 'extract',
 'key',
 'performance',
 'parameters',
 'Qualifications',
 'Job',
 'Location',
 'Locations',
 'Schedule',
 'Ability',
 'work',
 'large',
 'sets',
 'multiple',
 'sources',
 'Demonstrates',
 'proficiency',
 'several',
 'areas',
 'modeling',
 'machine',
 'learning',
 'algorithms',
 'Investigate',
 'problems',
 'identify',
 'patterns',
 'person',
 'love',
 'working',
 'parsing',
 'analyzing',
 'investigating',
 'interpreting',
 'modeling',
 'extracting',
 'Associate',
 'CTC',
 'opportunity',
 'qualified',
 'Associate',
 'Springfield',
 'VA',
 'Responsibilities',
 'includ

In [215]:
%matplotlib inline 


In [233]:
from nltk.probability import FreqDist
junior_list = sorted(junior_words, key = junior_words.count, reverse=True)
fdist_junior = FreqDist(junior_list)
junior_df = df.from_dict(fdist_junior, orient='index', dtype=int)
junior_df.columns = [ 'value']
jun = junior_df.sort_values(by = 'value', axis=0, ascending=False)

In [246]:
mid_low_list = sorted(mid_low_words, key = mid_low_words.count, reverse=True)
fdist_mid_low = FreqDist(mid_low_list)
mid_low_df = df.from_dict(fdist_mid_low, orient='index', dtype=int)
mid_low_df.columns = [ 'value']
mid_low_df.sort_values(by = 'value', axis=0, ascending=False)

,value
Experience,10
sets,9
large,8
team,7
SiteLock,6
position,6
quality,6
computer,6
hard,6
development,6


In [218]:
mid_high_list = sorted(mid_high_words, key = mid_high_words.count, reverse=True)
fdist_mid_high = FreqDist(mid_high_list)
mid_high_df = df.from_dict(fdist_mid_high, orient='index', dtype=int)
mid_high_df.columns = [ 'value']
mid_high_df.sort_values(by = 'value', axis=0, ascending=False)

,value
support,10
scientists,10
analytics,10
experience,10
team,9
big,8
engineers,8
business,8
test,7
Experience,7


In [219]:
senior_list = sorted(senior_words, key = senior_words.count, reverse=True)
fdist_senior = FreqDist(senior_list)
senior_df = df.from_dict(fdist_senior, orient='index', dtype=int)
senior_df.columns = [ 'value']
senior_df.sort_values(by = 'value', axis=0, ascending=False)

,value
experience,18
analysis,15
years,11
business,10
projects,9
looking,9
working,7
processing,7
solutions,6
Oculus,6


Now lets have a look at bigrams and trigrams as they are more meaningful. In the next few steps I am going to highlight most common bigrams and trigrams for each of 4 categories:

In [223]:
jun_pairs = [ " ".join(pair) for pair in nltk.bigrams(junior_words)]
jun_trigrams = sorted([ " ".join(pair) for pair in nltk.trigrams(junior_words)])
fdist_junpairs = FreqDist(jun_pairs)
fdist_jun_trigrams = FreqDist(jun_trigrams)


jun_bigrams = df.from_dict(fdist_junpairs, orient='index', dtype=int)
jun_bigrams.columns = [ 'value']
jun_bigrams.sort_values(by = 'value', axis=0, ascending=False)


#jun_trigs= df.from_dict(fdist_jun_trigrams, orient='index', dtype=int)
#jun_trigs.columns = [ 'value']
#jun_trigs.sort_values(by = 'value', axis=0, ascending=False)

,value
CLINICAL LAB,6
performance parameters,3
systems Proficient,3
learning algorithms,3
VA Responsibilities,3
love working,3
Machine learning,3
Associate Springfield,3
progress reports,3
summary findings,3


In [227]:
mid_low_pairs = [ " ".join(pair) for pair in nltk.bigrams(mid_low_words)]
mid_low_trigrams = sorted([ " ".join(pair) for pair in nltk.trigrams(mid_low_words)])
fdist_mid_low = FreqDist(mid_low_pairs)
fdist_mid_low_trigrams = FreqDist(mid_low_trigrams)

#mid_low_bigrams = df.from_dict(fdist_mid_low, orient='index', dtype=int)
#mid_low_bigrams.columns = [ 'value']
#mid_low_bigrams.sort_values(by = 'value', axis=0, ascending=False)


mid_low_trigs= df.from_dict(fdist_mid_low_trigrams, orient='index', dtype=int)
mid_low_trigs.columns = [ 'value']
mid_low_trigs.sort_values(by = 'value', axis=0, ascending=False)

,value
requires Handling sensitive,3
sets Partner teams,3
SiteLock join SiteLock,3
project needs Working,3
regular continuing position,3
field product performance,3
target specific markets,3
continuing position requires,3
engineers Generate laboratory,3
role utilize multiple,3


In [229]:
mid_high_pairs = [ " ".join(pair) for pair in nltk.bigrams(mid_high_words)]
mid_high_trigrams = sorted([ " ".join(pair) for pair in nltk.trigrams(mid_high_words)])
fdist_mid_high = FreqDist(mid_high_pairs)
fdist_mid_high_trigrams = FreqDist(mid_high_trigrams)


mid_high_bigrams = df.from_dict(fdist_mid_high, orient='index', dtype=int)
mid_high_bigrams.columns = [ 'value']
mid_high_bigrams.sort_values(by = 'value', axis=0, ascending=False)


#mid_high_trigs= df.from_dict(fdist_mid_high_trigrams, orient='index', dtype=int)
#mid_high_trigs.columns = [ 'value']
#mid_high_trigs.sort_values(by = 'value', axis=0, ascending=False)

,value
informed culture,3
merge sources,3
Senior experience,3
plans test,3
Product Development,3
leverage latest,3
engineers scientists,3
Amyx seeking,3
outside Office,3
Analytics Day,3


In [230]:
senior_pairs = [ " ".join(pair) for pair in nltk.bigrams(senior_words)]
senior_trigrams = sorted([ " ".join(pair) for pair in nltk.trigrams(senior_words)])
fdist_senior = FreqDist(senior_pairs)
fdist_senior_trigrams = FreqDist(senior_trigrams)

senior_bigrams = df.from_dict(fdist_senior, orient='index', dtype=int)
senior_bigrams.columns = [ 'value']
senior_bigrams.sort_values(by = 'value', axis=0, ascending=False)


#senior_trigs= df.from_dict(fdist_senior_trigrams, orient='index', dtype=int)
#senior_trigs.columns = [ 'value']
#senior_trigs.sort_values(by = 'value', axis=0, ascending=False)

,value
years experience,6
machine learning,5
processing analysis,5
Park CA,3
science projects,3
proficient generating,3
highly motivated,3
experience Mine,3
Use distributed,3
SQL NoSQL,3


Now it seems that towards senior level words as "expert", "management", "lead" appear more frequently. While words like "parsing" and "extracting" are more common for junior category

# Modelling

In [231]:
import sklearn.preprocessing
import sklearn.feature_extraction
tfidf = sklearn.feature_extraction.text.TfidfVectorizer()

In [251]:
words = pd.DataFrame({ 'words' : pd.Categorical(["well", "responsible", "associate", "analyzing", "models",
                                                 "extracting", "development", "performance", "test", "clinical", "scientists",
                                                 "lab", "large", "develop", "extract", "modeling", "measure", "amino", "learning",
                                                 "analysis", "tasks", "summary", "surveys", "experience", "sets", "team", 
                                                 "business", "projects", "quality", "expert", "big"]),
                'junior' : pd.Series((1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0)),
                'mid_low': pd.Series((0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,1,0,0)),
            'mid_high' :   pd.Series((0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1)),
                'senior' : pd.Series((0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,1,1))
                     })
words

,junior,mid_high,mid_low,senior,words
0,1,0,0,0,well
1,1,0,0,0,responsible
2,1,0,0,0,associate
3,1,0,0,0,analyzing
4,1,0,0,0,models
5,1,0,0,0,extracting
6,1,0,0,0,development
7,1,0,0,0,performance
8,1,0,0,0,test
9,1,0,0,0,clinical


In [236]:
#words_draft = pd.DataFrame({ 'words' : pd.Categorical([jun.index[:,30]]),
#                'junior' : pd.Series(1, index=list(range(30)), dtype='int32'),
#            'mid_low' :    pd.Series(0, index=list(range(30))),
#            'mid_high' :   pd.Series(0, index=list(range(30))),
#                'senior' : pd.Series(0, index=list(range(30)))
#                     })

In [255]:
logreg = sklearn.linear_model.LogisticRegression()
X = tfidf.fit_transform(words.words)
logreg.fit(X, words.junior)
list(zip(tfidf.get_feature_names(), logreg.coef_[0]))

[(u'amino', 0.26201299167670072),
 (u'analysis', 0.26201299167670072),
 (u'analyzing', 0.26201299167670072),
 (u'associate', 0.26201299167670072),
 (u'big', -0.55452963081652551),
 (u'business', -0.55452963081652551),
 (u'clinical', 0.26201299167670072),
 (u'develop', 0.26201299167670072),
 (u'development', 0.26201299167670072),
 (u'experience', -0.55452963081652551),
 (u'expert', -0.55452963081652551),
 (u'extract', 0.26201299167670072),
 (u'extracting', 0.26201299167670072),
 (u'lab', 0.26201299167670072),
 (u'large', -0.55452963081652551),
 (u'learning', 0.26201299167670072),
 (u'measure', 0.26201299167670072),
 (u'modeling', 0.26201299167670072),
 (u'models', 0.26201299167670072),
 (u'performance', 0.26201299167670072),
 (u'projects', -0.55452963081652551),
 (u'quality', -0.55452963081652551),
 (u'responsible', 0.26201299167670072),
 (u'scientists', 0.26201299167670072),
 (u'sets', -0.55452963081652551),
 (u'summary', 0.26201299167670072),
 (u'surveys', 0.26201299167670072),
 (u'ta

In [258]:
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
(X_train, X_test, Y_train, Y_test) = train_test_split(
  words['words'],
  words['junior'],
  test_size=0.33
)